<a href="https://colab.research.google.com/github/Alanazi433/GenAI/blob/main/GenAI/HW5/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow requests


In [2]:
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras import layers, models
import re
import string


In [3]:
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras import layers, models
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Download and combine text data
urls = [
"https://www.gutenberg.org/files/1041/1041-0.txt",  # Hamlet
    "https://www.gutenberg.org/files/152/152-0.txt",    # Macbeth
    "https://www.gutenberg.org/files/1112/1112-0.txt",  # Othello
    "https://www.gutenberg.org/files/2265/2265-0.txt",  # A Midsummer Night's Dream
    "https://www.gutenberg.org/files/1787/1787-0.txt",  # King Lear
    "https://www.gutenberg.org/files/2264/2264-0.txt"   # The Tempest
]

all_text = ""
for url in urls:
    response = requests.get(url)
    all_text += response.text + "\n\n"

# Clean text
def clean_text(text):
    text = re.sub(r"[^A-Za-z0-9\s]", "", text)
    text = text.lower()
    return text

all_text = clean_text(all_text)

# Tokenize and create sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([all_text])
sequences = tokenizer.texts_to_sequences([all_text])[0]
sequence_length = 100

# Create input (X) and target (y) sequences
sequences = [sequences[i:i + sequence_length + 1] for i in range(len(sequences) - sequence_length)]
X, y = [], []
for seq in sequences:
    X.append(seq[:-1])
    y.append(seq[-1])

X = np.array(X)
y = np.array(y)
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100


In [4]:
def train_and_evaluate_model(num_layers, units, epochs=70):
    inputs = layers.Input(shape=(None,), dtype="int32")
    x = layers.Embedding(vocab_size, embedding_dim)(inputs)

    # Add LSTM layers with dropout for regularization
    for _ in range(num_layers - 1):
        x = layers.LSTM(units, return_sequences=True, dropout=0.4)(x)
    x = layers.LSTM(units, dropout=0.4)(x)

    # Layer normalization
    x = layers.LayerNormalization()(x)

    outputs = layers.Dense(vocab_size, activation="softmax")(x)
    model = models.Model(inputs, outputs)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Learning rate scheduler for smoother convergence
    lr_scheduler = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=5, min_lr=1e-6)

    # Train with modified batch size or increased sequence length
    history = model.fit(X, y, batch_size=128, epochs=epochs, callbacks=[lr_scheduler])

    return model, history

# Configurations for models with different layers and units
configurations = [
    {"num_layers": 1, "units": 64},
    {"num_layers": 1, "units": 128},
    {"num_layers": 1, "units": 256},
    {"num_layers": 2, "units": 64},
    {"num_layers": 2, "units": 128},
    {"num_layers": 2, "units": 256},
    {"num_layers": 3, "units": 64},
    {"num_layers": 3, "units": 128},
    {"num_layers": 3, "units": 256}
]

models_histories = []
for config in configurations:
    print(f"\nTraining model with {config['num_layers']} LSTM layers and {config['units']} units per layer...")
    model, history = train_and_evaluate_model(config['num_layers'], config['units'])
    models_histories.append((f"{config['num_layers']} layers, {config['units']} units", model, history))



Training model with 1 LSTM layers and 64 units per layer...
Epoch 1/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.0207 - loss: 7.5986 - learning_rate: 0.0010
Epoch 2/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0402 - loss: 6.7041 - learning_rate: 0.0010
Epoch 3/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0803 - loss: 6.1812 - learning_rate: 0.0010
Epoch 4/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.1150 - loss: 5.6305 - learning_rate: 0.0010
Epoch 5/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1536 - loss: 5.0703 - learning_rate: 0.0010
Epoch 6/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.1954 - loss: 4.5576 - learning_rate: 0.0010
Epoch 7/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.2391 - loss: 4.0976 - learning_rate: 0.0010
Epoch 8/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.2929 - loss: 3.6523 - learning_rate: 0.0010
Epoch 9/70
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/st

In [5]:
def generate_text(model, tokenizer, seed_text, temperature=1.0, num_words=50):
    result = []
    input_text = seed_text

    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([input_text])[0]
        token_list = pad_sequences([token_list], maxlen=sequence_length, padding='pre')

        predictions = model.predict(token_list, verbose=0)[0]
        predictions = np.log(predictions + 1e-10) / temperature
        predictions = np.exp(predictions) / np.sum(np.exp(predictions))

        predicted_word_index = np.random.choice(range(vocab_size), p=predictions)
        predicted_word = tokenizer.index_word.get(predicted_word_index, '')

        input_text += " " + predicted_word
        result.append(predicted_word)

    return " ".join(result)


In [6]:
prompts = ["to be or not to be", "shall i compare thee to a summer's day", "all the world's a stage"]
temperatures = [0.1, 0.5, 1.0]

for model_name, model, _ in models_histories:
    print(f"\n--- Evaluating Model: {model_name} ---")
    for temp in temperatures:
        for prompt in prompts:
            print(f"\nPrompt: '{prompt}' | Temperature: {temp}")
            generated_text = generate_text(model, tokenizer, prompt, temperature=temp)
            print(f"Generated Text: {generated_text}")
            print("-" * 50)



--- Evaluating Model: 1 layers, 64 units ---

Prompt: 'to be or not to be' | Temperature: 0.1
 whoever hath
--------------------------------------------------

Prompt: 'shall i compare thee to a summer's day' | Temperature: 0.1
 to
--------------------------------------------------

Prompt: 'all the world's a stage' | Temperature: 0.1
 killing that loue thou hast
--------------------------------------------------

Prompt: 'to be or not to be' | Temperature: 0.5
 or
--------------------------------------------------

Prompt: 'shall i compare thee to a summer's day' | Temperature: 0.5
 all tongues that subject before the days watch in time
--------------------------------------------------

Prompt: 'all the world's a stage' | Temperature: 0.5
 heare the world may be here i doubt not all with ere it go to field be in prepare her to
--------------------------------------------------

Prompt: 'to be or not to be' | Temperature: 1.0
 
--------------------------------------------------

Prom